In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/unifesp-x-ray-body-part-classifier/sample_submission.csv
/kaggle/input/unifesp-x-ray-body-part-classifier/train.csv
/kaggle/input/unifesp-x-ray-body-part-classifier/test/test/1151/1.2.826.0.1.3680043.8.498.12729767832277330699945018998205716684/1.2.826.0.1.3680043.8.498.18989977955267938276912747789108046661/1.2.826.0.1.3680043.8.498.81525197438350521711990575810302547072-c.dcm
/kaggle/input/unifesp-x-ray-body-part-classifier/test/test/1151/1.2.826.0.1.3680043.8.498.12729767832277330699945018998205716684/1.2.826.0.1.3680043.8.498.22939084610457703998726890492710038593/1.2.826.0.1.3680043.8.498.69289037696158311261541536688493536871-c.dcm
/kaggle/input/unifesp-x-ray-body-part-classifier/test/test/7/1.2.826.0.1.3680043.8.498.72281060915288187045434950132399920411/1.2.826.0.1.3680043.8.498.47209177323312647360244968820032443496/1.2.826.0.1.3680043.8.498.32891312658070649342335208084432672152-c.dcm
/kaggle/input/unifesp-x-ray-body-part-classifier/test/test/843/1.2.826.0.1.368

Load in the relevant images for analysis

In [2]:
import pydicom
import torch
import matplotlib

In [3]:
# Load in the training data target values
train_df = pd.read_csv('../input/unifesp-x-ray-body-part-classifier/train.csv')